# Import Libraries

In [253]:
import pandas as pd
import numpy as np
import re

# Import Data

In [141]:
austin_intakes = pd.read_csv('./datasets/raw_data/austin_animal_center_intakes.csv')

In [142]:
austin_intakes.head(2)

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color
0,A786884,*Brock,01/03/2019 04:19:00 PM,January 2019,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor
1,A706918,Belle,07/05/2015 12:59:00 PM,July 2015,9409 Bluegrass Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,8 years,English Springer Spaniel,White/Liver


In [143]:
austin_outcomes = pd.read_csv('./datasets/raw_data/austin_animal_center_outcomes.csv')

In [144]:
austin_outcomes.head(2)

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A794011,Chunk,05/08/2019 06:20:00 PM,May 2019,05/02/2017,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
1,A776359,Gizmo,07/18/2018 04:02:00 PM,Jul 2018,07/12/2017,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown


In [145]:
austin_outcomes_2 = pd.read_csv('./datasets/raw_data/austin_animal_center_outcomes_2.csv')

In [146]:
austin_outcomes_2.head(2)

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A794011,Chunk,05/08/2019 06:20:00 PM,May 2019,05/02/2017,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
1,A776359,Gizmo,07/18/2018 04:02:00 PM,Jul 2018,07/12/2017,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown


# Clean Data

## Austin Intakes

In [147]:
# Alternative animals
austin_intakes[austin_intakes['Animal Type']=='Dog']['Animal Type'].value_counts()

Dog    76769
Name: Animal Type, dtype: int64

In [148]:
austin_intakes.shape

(136288, 12)

In [149]:
austin_intakes[austin_intakes['Animal Type']=='Dog'].head(3)

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color
0,A786884,*Brock,01/03/2019 04:19:00 PM,January 2019,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor
1,A706918,Belle,07/05/2015 12:59:00 PM,July 2015,9409 Bluegrass Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,8 years,English Springer Spaniel,White/Liver
2,A724273,Runster,04/14/2016 06:43:00 PM,April 2016,2818 Palomino Trail in Austin (TX),Stray,Normal,Dog,Intact Male,11 months,Basenji Mix,Sable/White


In [150]:
# Drop non-dogs
austin_intakes = austin_intakes[austin_intakes['Animal Type']=='Dog'].copy()

In [151]:
austin_intakes.shape

(76769, 12)

## Austin Outcomes

### Merge outcomes .csv

In [152]:
# merge austin outcomes data sets
austin_outcomes = pd.concat([austin_outcomes, austin_outcomes_2])

In [153]:
# drop duplicated rows
austin_outcomes.drop_duplicates(keep='first', inplace=True)

In [154]:
austin_outcomes.shape

(136564, 12)

In [155]:
austin_outcomes[austin_outcomes['Animal Type']!='Dog']['Animal Type'].value_counts()

Cat          51950
Other         7221
Bird           636
Livestock       25
Name: Animal Type, dtype: int64

In [156]:
# drop non-dogs
austin_outcomes = austin_outcomes[austin_outcomes['Animal Type']=='Dog'].copy()

## Merge Intakes & Outcomes

In [157]:
austin_intakes.shape

(76769, 12)

In [158]:
austin_outcomes.shape

(76732, 12)

In [159]:
# View duplicate intake Animal IDs
austin_intakes[austin_intakes['Animal ID'].duplicated(keep=False)==True].sort_values(by='Animal ID').head()

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color
91423,A006100,Scamp,03/07/2014 02:26:00 PM,March 2014,8700 Research in Austin (TX),Public Assist,Normal,Dog,Neutered Male,6 years,Spinone Italiano Mix,Yellow/White
20233,A006100,Scamp,12/07/2017 02:07:00 PM,December 2017,Colony Creek And Hunters Trace in Austin (TX),Stray,Normal,Dog,Neutered Male,10 years,Spinone Italiano Mix,Yellow/White
4344,A006100,Scamp,12/19/2014 10:21:00 AM,December 2014,8700 Research Blvd in Austin (TX),Public Assist,Normal,Dog,Neutered Male,7 years,Spinone Italiano Mix,Yellow/White
99487,A245945,Boomer,07/03/2014 05:55:00 PM,July 2014,Garden And Mildred in Austin (TX),Stray,Normal,Dog,Neutered Male,14 years,Labrador Retriever Mix,Tan
122656,A245945,Boomer,05/20/2015 10:34:00 PM,May 2015,7403 Blessing Ave in Austin (TX),Stray,Normal,Dog,Neutered Male,15 years,Labrador Retriever Mix,Tan


Duplciates appear to be same animal that has gone through intake at the shelter multiple times

In [160]:
# View duplicate outcome Animal IDs
austin_outcomes[austin_outcomes['Animal ID'].duplicated(keep=False)==True].sort_values(by='Animal ID').head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
101494,A006100,Scamp,12/07/2017 12:00:00 AM,Dec 2017,07/09/2007,Return to Owner,NaN,Dog,Neutered Male,10 years,Spinone Italiano Mix,Yellow/White
56686,A006100,Scamp,12/20/2014 04:35:00 PM,Dec 2014,07/09/2007,Return to Owner,NaN,Dog,Neutered Male,7 years,Spinone Italiano Mix,Yellow/White
115237,A006100,Scamp,03/08/2014 05:10:00 PM,Mar 2014,07/09/2007,Return to Owner,NaN,Dog,Neutered Male,6 years,Spinone Italiano Mix,Yellow/White
69529,A245945,Boomer,05/25/2015 11:49:00 AM,May 2015,05/23/2000,Transfer,Partner,Dog,Neutered Male,15 years,Labrador Retriever Mix,Tan
90697,A245945,Boomer,07/04/2014 03:26:00 PM,Jul 2014,05/23/2000,Return to Owner,NaN,Dog,Neutered Male,14 years,Labrador Retriever Mix,Tan


Duplciates appear to be same animal that has had recorded outcome at the shelter multiple times

In [161]:
austin_intakes.shape

(76769, 12)

In [162]:
austin_outcomes.shape


(76732, 12)

Rows of austin intakes and austin outcomes appear to largely line up, however dates for each animal are not necessarily listed in the same order and intakes has ~40 more rows

In [163]:
austin_outcomes.sort_values(by='Animal ID').head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
101494,A006100,Scamp,12/07/2017 12:00:00 AM,Dec 2017,07/09/2007,Return to Owner,NaN,Dog,Neutered Male,10 years,Spinone Italiano Mix,Yellow/White
115237,A006100,Scamp,03/08/2014 05:10:00 PM,Mar 2014,07/09/2007,Return to Owner,NaN,Dog,Neutered Male,6 years,Spinone Italiano Mix,Yellow/White
56686,A006100,Scamp,12/20/2014 04:35:00 PM,Dec 2014,07/09/2007,Return to Owner,NaN,Dog,Neutered Male,7 years,Spinone Italiano Mix,Yellow/White
39142,A047759,Oreo,04/07/2014 03:12:00 PM,Apr 2014,04/02/2004,Transfer,Partner,Dog,Neutered Male,10 years,Dachshund,Tricolor
81491,A134067,Bandit,11/16/2013 11:54:00 AM,Nov 2013,10/16/1997,Return to Owner,NaN,Dog,Neutered Male,16 years,Shetland Sheepdog,Brown/White


In [164]:
austin_intakes.sort_values(by='Animal ID').head()

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color
20233,A006100,Scamp,12/07/2017 02:07:00 PM,December 2017,Colony Creek And Hunters Trace in Austin (TX),Stray,Normal,Dog,Neutered Male,10 years,Spinone Italiano Mix,Yellow/White
91423,A006100,Scamp,03/07/2014 02:26:00 PM,March 2014,8700 Research in Austin (TX),Public Assist,Normal,Dog,Neutered Male,6 years,Spinone Italiano Mix,Yellow/White
4344,A006100,Scamp,12/19/2014 10:21:00 AM,December 2014,8700 Research Blvd in Austin (TX),Public Assist,Normal,Dog,Neutered Male,7 years,Spinone Italiano Mix,Yellow/White
71055,A047759,Oreo,04/02/2014 03:55:00 PM,April 2014,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,10 years,Dachshund,Tricolor
96671,A134067,Bandit,11/16/2013 09:02:00 AM,November 2013,12034 Research Blvd in Austin (TX),Public Assist,Injured,Dog,Neutered Male,16 years,Shetland Sheepdog,Brown/White


In [165]:
# Identify missing outcome Animal IDs
in_id = set(austin_intakes['Animal ID'])
out_id = set(austin_outcomes['Animal ID'])

len(list(out_id.difference(in_id)))

381

Missing intake data on 381 Animal IDs from outcomes

In [166]:
#List of all Animal IDs in austin outcomes and intakes
austin_in_out_ids = list(in_id.union(out_id))

In [167]:
# A few dogs are in the system a lot
austin_intakes['Animal ID'].value_counts().sort_values(ascending=False).head(10)

A721033    33
A718223    14
A718877    12
A706536    11
A761266     9
A737814     9
A717053     9
A700407     9
A716018     9
A616444     9
Name: Animal ID, dtype: int64

### Convert DateTimes

In [170]:
austin_intakes['DateTime'] = pd.to_datetime(austin_intakes['DateTime'])

In [172]:
austin_intakes.sort_values(by='DateTime').head(3)

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color
72982,A521520,Nina,2013-10-01 07:51:00,October 2013,Norht Ec in Austin (TX),Stray,Normal,Dog,Spayed Female,7 years,Border Terrier/Border Collie,White/Tan
117677,A664233,Stevie,2013-10-01 08:53:00,October 2013,7405 Springtime in Austin (TX),Stray,Injured,Dog,Intact Female,3 years,Pit Bull Mix,Blue/White
4788,A664234,NaN,2013-10-01 10:37:00,October 2013,5400 Jimmy Clay in Austin (TX),Stray,Injured,Dog,Intact Male,8 years,Border Collie Mix,Black/White


In [173]:
austin_outcomes['DateTime'] = pd.to_datetime(austin_outcomes['DateTime'])

In [174]:
austin_outcomes.head(3)

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
1,A776359,Gizmo,2018-07-18 16:02:00,Jul 2018,07/12/2017,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown
3,A720371,Moose,2016-02-13 17:59:00,Feb 2016,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
5,A659412,Princess,2020-10-05 14:37:00,Oct 2020,03/24/2013,Adoption,NaN,Dog,Spayed Female,7 years,Chihuahua Shorthair Mix,Brown


## Merge Intake & Outcome Frames

In [175]:
austin_intakes.head(3)

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color
0,A786884,*Brock,2019-01-03 16:19:00,January 2019,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor
1,A706918,Belle,2015-07-05 12:59:00,July 2015,9409 Bluegrass Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,8 years,English Springer Spaniel,White/Liver
2,A724273,Runster,2016-04-14 18:43:00,April 2016,2818 Palomino Trail in Austin (TX),Stray,Normal,Dog,Intact Male,11 months,Basenji Mix,Sable/White


In [176]:
austin_intakes.drop(columns=['MonthYear', 'Animal Type'], inplace=True)
austin_outcomes.drop(columns=['MonthYear', 'Animal Type'], inplace=True)

Sort intakes & outcomes

In [179]:
austin_intakes.sort_values(by=['Animal ID', 'DateTime'], inplace=True)
austin_intakes.reset_index(drop=True, inplace=True)

In [180]:
austin_outcomes.sort_values(by=['Animal ID', 'DateTime'], inplace=True)
austin_outcomes.reset_index(drop=True, inplace=True)

In [181]:
austin_intakes.head(3)

,Animal ID,Name,DateTime,Found Location,Intake Type,Intake Condition,Sex upon Intake,Age upon Intake,Breed,Color
0,A006100,Scamp,2014-03-07 14:26:00,8700 Research in Austin (TX),Public Assist,Normal,Neutered Male,6 years,Spinone Italiano Mix,Yellow/White
1,A006100,Scamp,2014-12-19 10:21:00,8700 Research Blvd in Austin (TX),Public Assist,Normal,Neutered Male,7 years,Spinone Italiano Mix,Yellow/White
2,A006100,Scamp,2017-12-07 14:07:00,Colony Creek And Hunters Trace in Austin (TX),Stray,Normal,Neutered Male,10 years,Spinone Italiano Mix,Yellow/White


In [182]:
austin_outcomes.head(3)

,Animal ID,Name,DateTime,Date of Birth,Outcome Type,Outcome Subtype,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A006100,Scamp,2014-03-08 17:10:00,07/09/2007,Return to Owner,NaN,Neutered Male,6 years,Spinone Italiano Mix,Yellow/White
1,A006100,Scamp,2014-12-20 16:35:00,07/09/2007,Return to Owner,NaN,Neutered Male,7 years,Spinone Italiano Mix,Yellow/White
2,A006100,Scamp,2017-12-07 00:00:00,07/09/2007,Return to Owner,NaN,Neutered Male,10 years,Spinone Italiano Mix,Yellow/White


In [183]:
# count ins and outs
in_count = austin_intakes['Animal ID'].value_counts()
out_count = austin_outcomes['Animal ID'].value_counts()
repeat_in = in_count[in_count > 1]
repeat_out = out_count[out_count > 1]

In [184]:
# separate ins
repeat_in = austin_intakes[austin_intakes['Animal ID'].isin(list(repeat_in.index))].copy(deep=True)
solo_in = austin_intakes[~austin_intakes['Animal ID'].isin(repeat_in['Animal ID'].unique())].copy(deep=True)

In [185]:
# separate outs
repeat_out = austin_outcomes[austin_outcomes['Animal ID'].isin(list(repeat_out.index))].copy(deep=True)
solo_out = austin_outcomes[~austin_outcomes['Animal ID'].isin(repeat_out['Animal ID'].unique())].copy(deep=True)

In [186]:
solo_in.shape[0] - solo_in['Animal ID'].isin(solo_out['Animal ID']).sum()

440

Missing outcome data on only 440 dogs that appeared once in the sets

### Merge Repeats

In [187]:
repeat_in.shape[0] - repeat_out.shape[0]    # probably only missing a few observations

62

Drop redundant cols

In [188]:
repeat_out.drop(columns=['Name', 'Breed', 'Color'], inplace=True)

In [189]:
repeat_out.rename(columns={
    'Animal ID': 'Animal ID',
    'DateTime': 'date_out',
    'Date of Birth': 'dob',
    'Outcome Type': 'outcome',
    'Outcome Subtype': 'outcome_sub',
    'Sex upon Outcome': 'sex_out',
    'Age upon Outcome': 'age_out'
}, inplace=True)

In [190]:
# storage
in_hold = repeat_in.copy()
in_hold.set_index('Animal ID', inplace=True)
out_hold = repeat_out.copy()                                                                
out_hold.set_index('Animal ID', inplace=True)

# drop missing IDs
in_hold.drop(list(set(in_hold.index).difference(set(out_hold.index))), inplace=True)
out_hold.drop(list(set(out_hold.index).difference(set(in_hold.index))), inplace=True)

# iterate
for i in repeat_in['Animal ID'].unique():                                                   # select ids of intake animals
    if i in set(repeat_out['Animal ID']):
        ins = repeat_in[repeat_in['Animal ID'] == i]                                        # grab intake rows
        outs = repeat_out[repeat_out['Animal ID'] == i]                                     # grab outcome rows

        if ins.shape[0] != outs.shape[0]:                                                   # check if ins & outs match up
            if i in set(in_hold.index):
                in_hold.drop(i, inplace=True)
            if i in set(out_hold.index):
                out_hold.drop(i, inplace=True)

Merge repeat visitors

In [191]:
repeat = pd.merge(left=in_hold, right=out_hold, how='left', on='Animal ID')
repeat.reset_index(drop=False, inplace=True)

Inspect repeat visitors

In [194]:
repeat[(repeat['date_out'] - repeat['DateTime']) < pd.Timedelta(0, units='minute')].head(7)

,Animal ID,Name,DateTime,Found Location,Intake Type,Intake Condition,Sex upon Intake,Age upon Intake,Breed,Color,date_out,dob,outcome,outcome_sub,sex_out,age_out
3,A006100,Scamp,2014-12-19 10:21:00,8700 Research Blvd in Austin (TX),Public Assist,Normal,Neutered Male,7 years,Spinone Italiano Mix,Yellow/White,2014-03-08 17:10:00,07/09/2007,Return to Owner,NaN,Neutered Male,6 years
6,A006100,Scamp,2017-12-07 14:07:00,Colony Creek And Hunters Trace in Austin (TX),Stray,Normal,Neutered Male,10 years,Spinone Italiano Mix,Yellow/White,2014-03-08 17:10:00,07/09/2007,Return to Owner,NaN,Neutered Male,6 years
7,A006100,Scamp,2017-12-07 14:07:00,Colony Creek And Hunters Trace in Austin (TX),Stray,Normal,Neutered Male,10 years,Spinone Italiano Mix,Yellow/White,2014-12-20 16:35:00,07/09/2007,Return to Owner,NaN,Neutered Male,7 years
8,A006100,Scamp,2017-12-07 14:07:00,Colony Creek And Hunters Trace in Austin (TX),Stray,Normal,Neutered Male,10 years,Spinone Italiano Mix,Yellow/White,2017-12-07 00:00:00,07/09/2007,Return to Owner,NaN,Neutered Male,10 years
11,A245945,Boomer,2015-05-20 22:34:00,7403 Blessing Ave in Austin (TX),Stray,Normal,Neutered Male,15 years,Labrador Retriever Mix,Tan,2014-07-04 15:26:00,05/23/2000,Return to Owner,NaN,Neutered Male,14 years
15,A287017,Stitch,2015-08-16 12:19:00,6620 Deatonhill Dr in Austin (TX),Stray,Normal,Spayed Female,14 years,Chihuahua Shorthair Mix,Black/White,2014-12-12 16:49:00,05/13/2001,Return to Owner,NaN,Spayed Female,13 years
19,A307010,Cooper,2017-06-01 11:47:00,806 Castle Ridge in Austin (TX),Owner Surrender,Normal,Neutered Male,14 years,Beagle Mix,Tricolor,2016-09-28 09:36:00,03/04/2003,Return to Owner,NaN,Neutered Male,13 years


Looks like we have ~17,000 observations of repeat animal IDs that show up in system and do not have a clear intake date vs outcome date. Will drop these observations so they do not muddy our predictions.

In [195]:
(repeat[(repeat['date_out'] - repeat['DateTime']) > pd.Timedelta(0, units='minute')]['date_out'] - repeat[(repeat['date_out'] - repeat['DateTime']) > pd.Timedelta(0, units='minute')]['DateTime']).mean()

Timedelta('171 days 19:01:19.788549982')

Average time for a repeat animal ID is 171 days. We will see later that the average time a dog is in the system that has only been in the system once is only 17 days. We're clearly having merging issues. If the original data had unique visit IDs, we could resolve this easily, but because time is a valuable resource, we will sadly have to drop the observations.

### Merge Solos

Drop missing outcomes for one time visitors

In [196]:
solo_in = solo_in[solo_in['Animal ID'].isin(solo_out['Animal ID'])].copy(deep=True)

Drop redundant columns

In [197]:
solo_out.drop(columns=['Name', 'Breed', 'Color'], inplace=True)

In [198]:
solo_out.rename(columns={
    'Animal ID': 'Animal ID',
    'DateTime': 'date_out',
    'Date of Birth': 'dob',
    'Outcome Type': 'outcome',
    'Outcome Subtype': 'outcome_sub',
    'Sex upon Outcome': 'sex_out',
    'Age upon Outcome': 'age_out'
}, inplace=True)

In [199]:
solo = pd.merge(left=solo_in, right=solo_out, how='left', on='Animal ID')

In [200]:
solo[(solo['DateTime'] > solo['date_out'])].head()

,Animal ID,Name,DateTime,Found Location,Intake Type,Intake Condition,Sex upon Intake,Age upon Intake,Breed,Color,date_out,dob,outcome,outcome_sub,sex_out,age_out
578,A531969,Black,2015-11-13 12:18:00,Outside Jurisdiction,Owner Surrender,Normal,Neutered Male,7 years,Pit Bull Mix,Black/White,2015-11-13 00:00:00,03/06/2008,Return to Owner,NaN,Neutered Male,7 years
1619,A624094,Bootz,2021-04-19 08:27:00,5901 East Stassney in Austin (TX),Owner Surrender,Sick,Intact Female,10 years,Pit Bull Mix,Brown Brindle/White,2021-04-19 00:00:00,10/10/2010,Euthanasia,Suffering,Intact Female,10 years
2360,A657016,Trap,2016-05-19 14:25:00,2600 Scofield Ridge Pkwy in Travis (TX),Stray,Injured,Intact Male,3 years,Pit Bull Mix,Black/White,2016-05-19 00:00:00,12/22/2012,Died,Enroute,Intact Male,3 years
5407,A672696,Oreo,2014-02-15 12:30:00,2 Brett Cove in Rollingwood (TX),Stray,Normal,Neutered Male,12 years,Rat Terrier Mix,Tricolor,2014-02-15 11:12:00,02/15/2002,Return to Owner,NaN,Neutered Male,12 years
6047,A674638,Sam,2014-03-16 11:46:00,Rollingwood & Bee Cave Rd in Austin (TX),Stray,Normal,Neutered Male,7 years,Labrador Retriever Mix,Chocolate,2014-03-16 10:48:00,03/16/2007,Return to Owner,NaN,Neutered Male,7 years


None of the animals returned the same day were in the system for longer than 1 day.

In [201]:
((solo[(solo['DateTime'] > solo['date_out'])])['date_out'] - (solo[(solo['DateTime'] > solo['date_out'])]['DateTime']) > pd.Timedelta(-1, units='days')).sum()

0

In [202]:
(solo[(solo['DateTime'] > solo['date_out'])])['date_out'] - (solo[(solo['DateTime'] > solo['date_out'])])['DateTime']

578     -1 days +11:42:00
1619    -1 days +15:33:00
2360    -1 days +09:35:00
5407    -1 days +22:42:00
6047    -1 days +23:02:00
               ...       
51541   -1 days +12:31:00
52162   -1 days +19:52:00
52163   -1 days +20:02:00
52670   -1 days +10:54:00
55129   -1 days +20:43:00
Length: 169, dtype: timedelta64[ns]

Check for correct time syncing

In [266]:
solo[solo['DateTime'] < solo['date_out']]

,Animal ID,Name,DateTime,Found Location,Intake Type,Intake Condition,Sex upon Intake,Age upon Intake,Breed,Color,date_out,dob,outcome,outcome_sub,sex_out,age_out
0,A047759,Oreo,2014-04-02 15:55:00,Austin (TX),Owner Surrender,Normal,Neutered Male,10 years,Dachshund,Tricolor,2014-04-07 15:12:00,04/02/2004,Transfer,Partner,Neutered Male,10 years
1,A134067,Bandit,2013-11-16 09:02:00,12034 Research Blvd in Austin (TX),Public Assist,Injured,Neutered Male,16 years,Shetland Sheepdog,Brown/White,2013-11-16 11:54:00,10/16/1997,Return to Owner,NaN,Neutered Male,16 years
2,A141142,Bettie,2013-11-16 14:46:00,Austin (TX),Stray,Aged,Spayed Female,15 years,Labrador Retriever/Pit Bull,Black/White,2013-11-17 11:40:00,06/01/1998,Return to Owner,NaN,Spayed Female,15 years
3,A163459,Sasha,2014-11-14 15:11:00,Ih 35 And 41St St in Austin (TX),Stray,Normal,Intact Female,15 years,Miniature Schnauzer Mix,Black/Gray,2014-11-14 19:28:00,10/19/1999,Return to Owner,NaN,Intact Female,15 years
4,A165752,Pep,2014-09-15 11:28:00,Gatlin Gun Rd And Brodie in Austin (TX),Stray,Normal,Neutered Male,15 years,Lhasa Apso Mix,Brown/White,2014-09-15 16:35:00,08/18/1999,Return to Owner,NaN,Neutered Male,15 years
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55385,A851186,Face,2022-02-08 11:06:00,6812 Hillcroft Dr in Austin (TX),Public Assist,Injured,Intact Male,2 years,Pit Bull,Brown,2022-02-08 18:16:00,02/08/2020,Return to Owner,NaN,Intact Male,2 years
55386,A851188,Static,2022-02-08 11:06:00,6812 Hillcroft Dr in Austin (TX),Public Assist,Normal,Intact Female,5 months,Pit Bull,Blue/White,2022-02-08 18:19:00,09/08/2021,Return to Owner,NaN,Intact Female,5 months
55387,A851191,Rose,2022-02-08 11:54:00,Austin (TX),Owner Surrender,Sick,Spayed Female,2 years,Doberman Pinsch Mix,Black,2022-02-08 14:32:00,04/08/2019,Euthanasia,Suffering,Spayed Female,2 years
55388,A851204,A851204,2022-02-08 14:11:00,8400 Jamestown Drive in Austin (TX),Stray,Normal,Intact Male,3 years,Dachshund Mix,Brown Brindle/White,2022-02-10 12:39:00,09/08/2018,Return to Owner,NaN,Intact Male,3 years


~55,000 observations

In [267]:
(solo[(solo['date_out'] - solo['DateTime']) > pd.Timedelta(0, units='minute')]['date_out'] - solo[(solo['date_out']   -   solo['DateTime']) > pd.Timedelta(0, units='minute')]['DateTime']).mean()

Timedelta('16 days 23:12:55.407098877')

### Drop Repeats

In [203]:
solo_in = set(austin_intakes['Animal ID'].value_counts()[austin_intakes['Animal ID'].value_counts() == 1].index)
solo_out = set(austin_outcomes['Animal ID'].value_counts()[austin_outcomes['Animal ID'].value_counts() == 1].index)

In [204]:
solo_ids = solo_in.intersection(solo_out)

In [205]:
austin_intakes = austin_intakes[austin_intakes['Animal ID'].isin(solo_ids)]
austin_intakes.set_index('Animal ID', inplace=True)
austin_outcomes = austin_outcomes[austin_outcomes['Animal ID'].isin(solo_ids)]
austin_outcomes.set_index('Animal ID', inplace=True)

In [206]:
austin_intakes.shape

(55394, 9)

In [207]:
austin_outcomes.shape

(55394, 9)

In [209]:
austin_intakes.rename(columns={
    'Name': 'name',
    'DateTime': 'date_in',
    'Found Location': 'location',
    'Intake Type': 'intake_type',
    'Intake Condition': 'condition',
    'Sex upon Intake': 'sex_in',
    'Age upon Intake': 'age_in',
    'Breed': 'breed',
    'Color': 'color'
}, inplace=True)

austin_outcomes.rename(columns={
    'Name': 'name',
    'DateTime': 'date_out',
    'Date of Birth': 'dob',
    'Outcome Type': 'outcome',
    'Outcome Subtype': 'outcome_sub',
    'Sex upon Outcome': 'sex_out',
    'Age upon Outcome': 'age_out',
    'Breed': 'breed',
    'Color': 'color'
}, inplace=True)

austin_outcomes.drop(columns=['name', 'breed', 'color'], inplace=True)

Final Merge

In [210]:
austin = pd.merge(left=austin_intakes, right=austin_outcomes, how='left', left_on='Animal ID', right_on='Animal ID')

In [211]:
austin.reset_index(drop=False, inplace=True)

In [212]:
austin.rename(columns={'Animal ID':'id'}, inplace=True)

### Drop NA

In [213]:
austin = austin[austin['outcome'].notna()].copy(deep=True)

### Impute Missing

In [214]:
austin['sex_in'].value_counts()

Intact Male      21083
Intact Female    19420
Neutered Male     7706
Spayed Female     6679
Unknown            497
Name: sex_in, dtype: int64

In [215]:
austin['sex_in'].fillna('Intact Male', inplace=True)

Filling with most frequent

In [216]:
austin['sex_out'].value_counts()

Neutered Male    21271
Spayed Female    19124
Intact Male       7518
Intact Female     6975
Unknown            497
Name: sex_out, dtype: int64

In [217]:
austin['sex_out'].fillna('Neutered Male', inplace=True)

Filling with most frequent

In [218]:
austin['age_out'].value_counts()[:5]

2 years     10072
1 year       9837
2 months     5527
3 years      3924
4 years      2297
Name: age_out, dtype: int64

In [219]:
austin['age_out'].fillna('1 year', inplace=True)

In [220]:
austin.reset_index(drop=True, inplace=True)

In [221]:
# convert month to decimil
austin.loc[austin[austin['age_out'].str.contains('month')].index.values, 'age_out'] = austin['age_out'][austin['age_out'].str.contains('month')].apply(lambda x: str(int(x.split()[0])/12))
austin.loc[austin[austin['age_in'].str.contains('month')].index.values, 'age_in'] = austin['age_in'][austin['age_in'].str.contains('month')].apply(lambda x: str(int(x.split()[0])/12))

In [222]:
# convert week to decimil
austin.loc[austin[austin['age_out'].str.contains('week')].index.values, 'age_out'] = austin['age_out'][austin['age_out'].str.contains('week')].apply(lambda x: str(int(x.split()[0])/52))
austin.loc[austin[austin['age_in'].str.contains('week')].index.values, 'age_in'] = austin['age_in'][austin['age_in'].str.contains('week')].apply(lambda x: str(int(x.split()[0])/52))

In [223]:
# convert day to decimil
austin.loc[austin[austin['age_out'].str.contains('day')].index.values, 'age_out'] = austin['age_out'][austin['age_out'].str.contains('day')].apply(lambda x: str(int(x.split()[0])/365))
austin.loc[austin[austin['age_in'].str.contains('day')].index.values, 'age_in'] = austin['age_in'][austin['age_in'].str.contains('day')].apply(lambda x: str(int(x.split()[0])/365))

In [224]:
# drop 'years'
austin.loc[austin[austin['age_out'].str.contains('year')].index.values, 'age_out'] = austin['age_out'][austin['age_out'].str.contains('year')].apply(lambda x: str(x.split()[0]))
austin.loc[austin[austin['age_in'].str.contains('year')].index.values, 'age_in'] = austin['age_in'][austin['age_in'].str.contains('year')].apply(lambda x: str(x.split()[0]))

In [225]:
# fix negative numbers
austin['age_out'] = austin['age_out'].str.replace('-', '')
austin['age_in'] = austin['age_in'].str.replace('-', '')

In [226]:
austin['age_out'] = round(austin['age_out'].astype(float), 2)
austin['age_in'] = round(austin['age_in'].astype(float), 2)

In [227]:
austin['sex_in'].value_counts()

Intact Male      21084
Intact Female    19420
Neutered Male     7706
Spayed Female     6679
Unknown            497
Name: sex_in, dtype: int64

In [228]:
austin['sex'] = austin['sex_in'].map({
    'Intact Male': 1,
    'Neutered Male': 1,
    'Intact Female': 0,
    'Spayed Female': 0,
    'Unknown': 1
})

In [229]:
austin['intact_in'] = austin['sex_in'].map({
    'Intact Male': 1,
    'Neutered Male': 0,
    'Intact Female': 1,
    'Spayed Female': 0,
    'Unknown': 1
})

In [230]:
austin['intact_out'] = austin['sex_out'].map({
    'Intact Male': 1,
    'Neutered Male': 0,
    'Intact Female': 1,
    'Spayed Female': 0,
    'Unknown': 1
})

In [231]:
austin['dob'] = pd.to_datetime(austin['dob'])

In [232]:
austin.drop(columns=['sex_in', 'sex_out'], inplace=True)

In [233]:
austin.head(3)

,id,name,date_in,location,intake_type,condition,age_in,breed,color,date_out,dob,outcome,outcome_sub,age_out,sex,intact_in,intact_out
0,A047759,Oreo,2014-04-02 15:55:00,Austin (TX),Owner Surrender,Normal,10.0,Dachshund,Tricolor,2014-04-07 15:12:00,2004-04-02,Transfer,Partner,10.0,1,0,0
1,A134067,Bandit,2013-11-16 09:02:00,12034 Research Blvd in Austin (TX),Public Assist,Injured,16.0,Shetland Sheepdog,Brown/White,2013-11-16 11:54:00,1997-10-16,Return to Owner,NaN,16.0,1,0,0
2,A141142,Bettie,2013-11-16 14:46:00,Austin (TX),Stray,Aged,15.0,Labrador Retriever/Pit Bull,Black/White,2013-11-17 11:40:00,1998-06-01,Return to Owner,NaN,15.0,0,0,0


In [300]:
austin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55382 entries, 0 to 55381
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           55382 non-null  object        
 1   name         43230 non-null  object        
 2   date_in      55382 non-null  datetime64[ns]
 3   location     55382 non-null  object        
 4   intake_type  55382 non-null  object        
 5   condition    55382 non-null  object        
 6   age_in       55382 non-null  float64       
 7   breed        55382 non-null  object        
 8   color        55382 non-null  object        
 9   date_out     55382 non-null  datetime64[ns]
 10  dob          55382 non-null  datetime64[ns]
 11  outcome      55382 non-null  object        
 12  outcome_sub  20689 non-null  object        
 13  age_out      55382 non-null  float64       
 14  sex          55382 non-null  int64         
 15  intact_in    55382 non-null  int64         
 16  inta

## Intake, Outcome, and Condition Classes

### Intake_subtype

In [236]:
austin['intake_type'].value_counts()

Stray                 40953
Owner Surrender        9582
Public Assist          4422
Abandoned               258
Euthanasia Request      170
Wildlife                  1
Name: intake_type, dtype: int64

In [237]:
austin['intake_type'] = austin['intake_type'].map(
    {
        'Owner Surrender':'surrender', 
        'Public Assist':'public_assist', 
        'Stray':'stray', 
        'Euthanasia Request':'euth_request',
        'Abandoned':'abandoned',
        'wildlife':'wildlife',
        }
    )

### Outcome_sub

In [238]:
austin['outcome_sub'].isna().sum()

34695

In [239]:
austin.drop(columns=['outcome_sub'], inplace=True)

### Condition

In [240]:
austin['condition'].value_counts()

Normal        48941
Injured        2862
Sick           1717
Nursing        1179
Aged            333
Other           112
Medical          93
Pregnant         63
Neonatal         49
Behavior         24
Feral             9
Med Urgent        2
Med Attn          2
Name: condition, dtype: int64

In [241]:
austin['condition'] = austin['condition'].map(
    {
        'Normal':'normal', 
        'Injured':'injured', 
        'Sick':'sick', 
        'Nursing':'nursing',
        'Aged': 'aged',
        'Other': 'other',
        'Medical': 'medical',
        'Pregnant': 'pregnant',
        'Neonatal': 'neonatal',
        'Behavior': 'behavior',
        'Med Urgent':'injured',
        'Med Attn ':'injured',
        }
    )

### Outcome

In [242]:
austin['outcome'].value_counts()

Adoption           25609
Transfer           14581
Return to Owner    12922
Euthanasia          1588
Rto-Adopt            348
Died                 275
Disposal              44
Missing               19
Name: outcome, dtype: int64

In [243]:
austin['outcome'] = austin['outcome'].map(
    {
        'Adoption': 'adoption', 
        'Transfer': 'transfer', 
        'Return to Owner': 'return_owner', 
        'Euthanasia': 'euthanasia',
        'Rto-Adopt': 'return_owner',
        'Died': 'died',
        'Disposal': 'disposal',
        'Missing': 'missing',
        }
    )

## Age corrections

In [244]:
austin['age_in'] = np.round(austin['age_in'])

In [245]:
austin['age_in'].value_counts()

0.0     15365
1.0     13617
2.0     10062
3.0      3862
4.0      2329
5.0      2231
6.0      1492
7.0      1315
8.0      1283
10.0     1138
9.0       753
12.0      522
11.0      432
13.0      342
14.0      259
15.0      201
16.0       96
17.0       45
18.0       23
19.0       14
20.0        3
23.0        1
24.0        1
Name: age_in, dtype: int64

In [246]:
austin['age_out'] = np.round(austin['age_out'])
austin['age_out'].value_counts()

0.0     14873
1.0     13918
2.0     10072
3.0      3924
4.0      2297
5.0      2259
6.0      1495
8.0      1341
7.0      1315
10.0     1160
9.0       732
12.0      543
11.0      445
13.0      360
14.0      262
15.0      204
16.0       99
17.0       45
18.0       23
19.0       14
20.0        3
23.0        1
24.0        1
Name: age_out, dtype: int64

### Add age column

In [247]:
age_cats = {
    0: 'Baby',
    1: 'Young',
    2: 'Adult', 3: 'Adult', 4: 'Adult', 5: 'Adult', 6: 'Adult', 7: 'Adult', 8: 'Adult',
    9: 'Senior', 10: 'Senior', 11: 'Senior', 12: 'Senior', 13: 'Senior', 14: 'Senior', 15: 'Senior', 16: 'Senior', 
    17: 'Senior', 18: 'Senior', 19: 'Senior', 20: 'Senior', 21: 'Senior', 22: 'Senior', 23: 'Senior', 24: 'Senior', 
    25: 'Senior', 26: 'Senior', 27: 'Senior', 28: 'Senior', 29: 'Senior', 30: 'Senior',
    np.nan: 'Adult' 
}

In [248]:
austin['age'] = austin['age_out'].map(age_cats)

In [249]:
austin['age'].value_counts()

Adult     22703
Baby      14873
Young     13918
Senior     3892
Name: age, dtype: int64

## Clean Color Columns

In [250]:
austin['color'].value_counts()

Black/White              6301
White                    3108
Brown/White              3102
Black                    3089
Tan/White                2831
                         ... 
Yellow Brindle/Blue         1
Red Merle/Tan               1
Chocolate/Red Tick          1
Chocolate/Brown Merle       1
Liver Tick/Brown            1
Name: color, Length: 368, dtype: int64

Unique list of dog colors

In [251]:
colors = list(set(austin['color']))

Build primary color & secondary color combos

In [254]:
primary_colors = []
secondary_colors = []

for i in range(len(austin['color'])):
    if str(austin['color'][i])!='nan':
        color_list = re.split(' |/', austin['color'][i])
        if len(color_list)==1:
            primary_colors.append(color_list[0])
            secondary_colors.append(color_list[0])
        else:
            primary_colors.append(color_list[0])
            secondary_colors.append(color_list[1])
    else:
        primary_colors.append(np.nan)
        secondary_colors.append(np.nan)   

In [255]:
austin['primary_color'] = primary_colors

In [256]:
austin['secondary_color'] = secondary_colors

In [260]:
austin.head(3)

,id,name,date_in,location,intake_type,condition,age_in,breed,color,date_out,dob,outcome,age_out,sex,intact_in,intact_out,age,primary_color,secondary_color
0,A047759,Oreo,2014-04-02 15:55:00,Austin (TX),surrender,normal,10.0,Dachshund,Tricolor,2014-04-07 15:12:00,2004-04-02,transfer,10.0,1,0,0,Senior,Tricolor,Tricolor
1,A134067,Bandit,2013-11-16 09:02:00,12034 Research Blvd in Austin (TX),public_assist,injured,16.0,Shetland Sheepdog,Brown/White,2013-11-16 11:54:00,1997-10-16,return_owner,16.0,1,0,0,Senior,Brown,White
2,A141142,Bettie,2013-11-16 14:46:00,Austin (TX),stray,aged,15.0,Labrador Retriever/Pit Bull,Black/White,2013-11-17 11:40:00,1998-06-01,return_owner,15.0,0,0,0,Senior,Black,White


Lowercase color columns

In [261]:
austin['primary_color'] = austin['primary_color'].str.lower()

In [262]:
austin['primary_color'].isna().sum()

0

In [268]:
austin['primary_color'].value_counts()[:10]

black        14932
brown        10295
white         9971
tan           6619
red           2624
blue          2510
tricolor      2427
chocolate     1307
cream          858
sable          754
Name: primary_color, dtype: int64

In [264]:
austin['secondary_color'] = austin['secondary_color'].str.lower()

In [265]:
austin['secondary_color'].isna().sum()

0

In [267]:
austin['secondary_color'].value_counts()[:10]

white       19881
black        8124
tan          7139
brown        7037
tricolor     2972
brindle      2734
red          1387
cream         854
merle         787
gray          715
Name: secondary_color, dtype: int64

# Export data

In [269]:
austin.to_csv('./datasets/working_data/austin.csv', index=False)